In [0]:
import itertools as it
import sys
import os

ServicePrincipalID = "05b0a1db-8ab9-4352-803a-94b3e82d3edc"
ServicePrincipalKey = "TYVw2WK4j~85o4bPMD-M68Bh8d-YHbD4Y."
DirectoryID = "b41b72d0-4e9f-4c26-8a69-f949f367c91d"
#Combine DirectoryID into full string
Directory = "https://login.microsoftonline.com/{}/oauth2/token".format(DirectoryID)

#Input parameter: file for transform
dbutils.widgets.text("fileName", "", "")
input_file_name = "CoordStatusMA_Short.csv"

#Storage account options
container_name = "covidsbcontainer"
stac_name = "covidsbstac2"

#DBFS
url = "abfss://{}@{}.dfs.core.windows.net/".format(container_name, stac_name)
mnt_path = '/mnt/covid'
input_file_path = "/dbfs" + mnt_path + "/input/" + input_file_name
output_file_path = "/dbfs" + mnt_path + "/output/" + os.path.splitext(os.path.basename(input_file_path))[0] + "_prepared.csv"

In [0]:
#Create configurations for our connection
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": ServicePrincipalID,
           "fs.azure.account.oauth2.client.secret": ServicePrincipalKey,
           "fs.azure.account.oauth2.client.endpoint": Directory}

#Mount
dbutils.fs.mount(
  source = url,
  mount_point = mnt_path,
  extra_configs = configs)

Out[2]: True

In [0]:
status = None
date = None
column_names=['Status','Date','Latitude','Longitude','Count']
if os.path.isfile(input_file_path):
  new_file = open(output_file_path, 'w')
  new_file.write(column_names[0] + "," + column_names[1] + "," + column_names[2] + "," + column_names[3] + "," + column_names[4] +"\n")
  with open(input_file_path, mode='r', encoding='cp1252') as f:
      for line in f.readlines():
          if len(line.strip()) > 0:
              if line.startswith('Status='):
                  status_line = line.split(',')[0]
                  status = status_line.split('=')[1]
              else:
                  #Get date
                  date = line.split(',')[0]
                  #Get list with Latitudes
                  _la = line.split(',')[1::3]
                  #Get list with Longitudes
                  _lo = line.split(',')[2::3]
                  #Get list with counts
                  _c = line.split(',')[3::3]
                  if len(_la) == len(_lo) == len(_c):
                      #Aggregate elements from lists
                      for (la, lo, c) in zip(_la, _lo, _c):
                          #while lists are not empty
                          if len(la) > 0 and len(lo) > 0 and len(c) > 0:
                              new_file.write(status.strip() + "," + date.strip() + "," + la.strip() + "," + lo.strip() + "," + c.strip() +"\n")
  new_file.close()

In [0]:
dbutils.fs.unmount(mnt_path)

/mnt/covid has been unmounted.
Out[4]: True